# Workshop 3: AI Agents with Azure AI Foundry

Welcome to Workshop 3! In this notebook, you'll learn how to build intelligent AI agents using Azure AI Foundry's Agent Service and function calling capabilities.

## What You'll Learn

1. **Agent Fundamentals** - Understanding AI agents and their capabilities
2. **Azure AI Agent Service** - Using the cloud-native agent platform
3. **Function Calling** - Enabling agents to use tools and APIs
4. **Multi-Step Reasoning** - Building agents that can plan and execute
5. **Custom Tools** - Creating your own functions for agents
6. **Conversation Management** - Handling stateful agent interactions

## Prerequisites

- Completed Workshop 1 (Deploy Your First Model)
- Completed Workshop 2 (Tracing) - optional but recommended
- Azure AI Foundry project with Agent Service enabled
- Environment variables set up correctly

## Learning Objectives

By the end of this workshop, you will:
- Understand AI agent architecture and capabilities
- Build agents using Azure AI Foundry Agent Service
- Create custom tools and functions for agents
- Implement multi-step reasoning workflows
- Handle stateful conversations and memory
- Trace and monitor agent interactions

## 1. Environment Setup and Imports

Let's set up our environment for building AI agents.

In [1]:
import os
import json
import time
import asyncio
from typing import Dict, List, Any, Optional
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient

# OpenTelemetry imports for tracing (optional)
try:
    from opentelemetry import trace
    from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor
    from azure.core.settings import settings
    settings.tracing_implementation = "opentelemetry"
    OpenAIInstrumentor().instrument()
    print(" Tracing enabled for agent interactions")
except ImportError:
    print("ℹ Tracing not available - install opentelemetry packages for tracing")

# Load environment variables
load_dotenv()

print(" AI Agents Workshop Environment Check:")
print("-" * 50)

# Check required environment variables
required_vars = [
    'PROJECT_ENDPOINT',
    'AZURE_OPENAI_DEPLOYMENT_NAME'
]

for var in required_vars:
    value = os.getenv(var)
    status = "" if value else ""
    print(f"{status} {var}: {'Set' if value else '❌ Not set'}")

print("-" * 50)

 Tracing enabled for agent interactions
 AI Agents Workshop Environment Check:
--------------------------------------------------
 PROJECT_ENDPOINT: Set
 AZURE_OPENAI_DEPLOYMENT_NAME: Set
--------------------------------------------------


## 2. Understanding AI Agents

Before we build agents, let's understand what makes them different from simple AI interactions.

In [2]:
def explain_agent_concepts():
    """
    Explain key AI agent concepts and capabilities.
    """
    print(" AI Agent Concepts:")
    print("=" * 60)
    
    concepts = {
        " Autonomy": {
            "definition": "Agents can make decisions and take actions independently",
            "example": "Deciding which tool to use based on user request",
            "benefit": "Reduces need for explicit programming of every scenario"
        },
        " Tool Use": {
            "definition": "Agents can call external functions and APIs",
            "example": "Weather API, database queries, file operations",
            "benefit": "Extends agent capabilities beyond text generation"
        },
        " Planning": {
            "definition": "Agents can break down complex tasks into steps",
            "example": "Research → Analyze → Summarize → Present",
            "benefit": "Handles multi-step workflows automatically"
        },
        " Memory": {
            "definition": "Agents maintain context across interactions",
            "example": "Remembering previous conversation history and decisions",
            "benefit": "Enables coherent, stateful conversations"
        },
        " Iteration": {
            "definition": "Agents can retry and refine their approach",
            "example": "If first tool call fails, try alternative approach",
            "benefit": "More robust and adaptive behavior"
        }
    }
    
    for concept, details in concepts.items():
        print(f"\n{concept}")
        print(f"   Definition: {details['definition']}")
        print(f"   Example: {details['example']}")
        print(f"   Benefit: {details['benefit']}")
    
    print(f"\n Agent vs. Simple AI Chat:")
    print("")
    print(" Simple AI Chat       AI Agent            ")
    print("")
    print(" Single response      Multi-step planning ")
    print(" Text only            Can use tools       ")
    print(" Stateless            Maintains memory    ")
    print(" Reactive             Proactive           ")
    print(" Fixed capabilities   Extensible tools    ")
    print("")

explain_agent_concepts()

 AI Agent Concepts:

 Autonomy
   Definition: Agents can make decisions and take actions independently
   Example: Deciding which tool to use based on user request
   Benefit: Reduces need for explicit programming of every scenario

 Tool Use
   Definition: Agents can call external functions and APIs
   Example: Weather API, database queries, file operations
   Benefit: Extends agent capabilities beyond text generation

 Planning
   Definition: Agents can break down complex tasks into steps
   Example: Research → Analyze → Summarize → Present
   Benefit: Handles multi-step workflows automatically

 Memory
   Definition: Agents maintain context across interactions
   Example: Remembering previous conversation history and decisions
   Benefit: Enables coherent, stateful conversations

 Iteration
   Definition: Agents can retry and refine their approach
   Example: If first tool call fails, try alternative approach
   Benefit: More robust and adaptive behavior

 Agent vs. Simple AI Chat:


## 3. Connect to Azure AI Foundry Agent Service

Let's connect to Azure AI Foundry and initialize the Agent Service.

In [3]:
# Connect to Azure AI Foundry Agent Service
try:
    print(" Connecting to Azure AI Foundry Agent Service...")
    
    # Initialize project client
    credential = DefaultAzureCredential()
    project_client = AIProjectClient(
        endpoint=os.getenv('PROJECT_ENDPOINT'),
        credential=credential
    )
    
    # Initialize agents client - using the new direct initialization
    agents_client = AgentsClient(
        endpoint=os.getenv('PROJECT_ENDPOINT'),
        credential=credential
    )
    
    print(" Connected to Azure AI Foundry Agent Service")
    print(f" Project: {os.getenv('PROJECT_ENDPOINT')}")
    print(f" Model: {os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')}")
    
    # Test connection
    print(f"\n ✅ Connection successful!")
    print(f"    Project Client: ✅ Ready")
    print(f"    Agents Client: ✅ Ready")
    
except Exception as e:
    print(f" Connection ❌ failed: {e}")
    print(" Make sure your environment variables are set correctly")
    print(" Ensure you have the proper permissions for Azure AI Foundry")
    print(" Try running: az login")
    print(" Required packages: azure-ai-projects, azure-ai-agents")

 Connecting to Azure AI Foundry Agent Service...
 Connected to Azure AI Foundry Agent Service
 Project: https://foundry-fr8lmydf.services.ai.azure.com/api/projects/foundry-fr8lmydf-project
 Model: gpt-4.1-mini

 ✅ Connection successful!
    Project Client: ✅ Ready
    Agents Client: ✅ Ready


## 4. Create Your First AI Agent

Let's create a simple AI agent and test basic functionality.

In [4]:
# Create a simple AI agent
def create_simple_agent():
    """
    Create a basic AI agent for general assistance.
    """
    print(" Creating Simple AI Agent...")
    
    # Define agent instructions
    instructions = """
    You are a helpful AI assistant specialized in providing information and assistance.
    Be concise, accurate, and helpful in your responses.
    If you're unsure about something, say so clearly.
    """
    
    try:
        # Create the agent
        agent = agents_client.create_agent(
            model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
            name="Simple Assistant",
            instructions=instructions,
            description="A helpful AI assistant for general questions and tasks"
        )
        
        print(f" Agent ✅ created successfully!")
        print(f" Agent ID: {agent.id}")
        print(f" Name: {agent.name}")
        print(f" Model: {agent.model}")
        
        return agent
        
    except Exception as e:
        print(f" ❌ Failed to create agent: {e}")
        return None

# Create the agent
simple_agent = create_simple_agent()

 Creating Simple AI Agent...
 Agent ✅ created successfully!
 Agent ID: asst_pya4a1XOYl0TqZPKHdKFGf8h
 Name: Simple Assistant
 Model: gpt-4.1-mini


## 5. Test Agent Conversation

Let's create a conversation thread and interact with our agent.

In [5]:
# Test agent conversation
def test_agent_conversation(agent, user_message: str):
    """
    Simple agent conversation test.
    """
    if not agent:
        print(" No agent available for testing")
        return
    
    print(f" Testing: {user_message}")
    
    try:
        # Create thread and send message
        thread = agents_client.threads.create()
        agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_message
        )
        
        # Run agent
        run = agents_client.runs.create(
            thread_id=thread.id,
            agent_id=agent.id
        )
        
        # Wait for completion
        while run.status in ["queued", "in_progress"]:
            time.sleep(1)
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
        
        # Get response
        if run.status == "completed":
            messages = agents_client.messages.list(thread_id=thread.id)
            for msg in list(messages):
                if msg.role == "assistant":
                    print(f" {msg.content[0].text.value}")
                    return thread.id
        else:
            print(f" ❌ Failed: {run.status}")
            
    except Exception as e:
        print(f" ❌ Error: {e}")
        
    return None

test_agent_conversation(simple_agent, "Hello! Can you explain what AI agents are in simple terms?")


 Testing: Hello! Can you explain what AI agents are in simple terms?
 Hello! Sure!  

AI agents are computer programs designed to perform tasks or make decisions on their own. They perceive their surroundings through data or sensors, think about what to do, and then take actions to achieve specific goals.  

For example, a virtual assistant like Siri or Alexa is an AI agent that listens to your voice, understands your request, and then answers or helps you.  

In short, AI agents are like smart helpers that can understand, decide, and act. Would you like to know about any specific type of AI agent?


'thread_DEoBIrvtlQQ9e9PUVkhOoI90'

## 6. Function Calling and Custom Tools

Now let's create an agent with custom tools that can perform specific actions.

In [6]:
# Create custom tools for the agent
def create_calculator_tool():
    """
    Create a calculator tool definition.
    """
    return {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform basic mathematical calculations",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression to evaluate (e.g., '2 + 3 * 4')"
                    }
                },
                "required": ["expression"]
            }
        }
    }

def create_weather_tool():
    """
    Create a weather tool definition (mock).
    """
    return {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather information for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name or location"
                    },
                    "units": {
                        "type": "string",
                        "description": "Temperature units (celsius or fahrenheit)",
                        "enum": ["celsius", "fahrenheit"],
                        "default": "celsius"
                    }
                },
                "required": ["location"]
            }
        }
    }

# Create an agent with tools
def create_tool_agent():
    """
    Create an AI agent with custom tools.
    """
    print(" Creating AI Agent with Custom Tools...")
    
    # Define agent instructions
    instructions = """
    You are a helpful assistant with access to tools.
    You can perform calculations and get weather information.
    Always use the appropriate tool when the user asks for calculations or weather.
    Be helpful and explain what you're doing when using tools.
    """
    
    # Define tools
    tools = [
        create_calculator_tool(),
        create_weather_tool()
    ]
    
    try:
        # Create the agent with tools
        agent = agents_client.create_agent(
            model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
            name="Tool Agent",
            instructions=instructions,
            description="An AI assistant with calculator and weather tools",
            tools=tools
        )
        
        print(f" Tool agent ✅ created successfully!")
        print(f" Agent ID: {agent.id}")
        print(f" Name: {agent.name}")
        print(f" Tools: {len(tools)} tools available")
        print(f"   - Calculator for math operations")
        print(f"   - Weather lookup (mock)")
        
        return agent
        
    except Exception as e:
        print(f" ❌ Failed to create tool agent: {e}")
        return None

# Create the tool agent
tool_agent = create_tool_agent()

 Creating AI Agent with Custom Tools...
 Tool agent ✅ created successfully!
 Agent ID: asst_fKMNWOPuLYOtWpe4FEFDFB1d
 Name: Tool Agent
 Tools: 2 tools available
   - Calculator for math operations
   - Weather lookup (mock)


In [7]:
# Implement tool function handlers
def handle_tool_calls(tool_calls):
    """
    Handle tool calls from the agent.
    """
    results = []
    
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        print(f" Agent calling tool: {function_name}")
        print(f" Arguments: {function_args}")
        
        if function_name == "calculate":
            result = handle_calculate(function_args["expression"])
        elif function_name == "get_weather":
            result = handle_get_weather(
                function_args["location"], 
                function_args.get("units", "celsius")
            )
        else:
            result = f"Unknown function: {function_name}"
        
        results.append({
            "tool_call_id": tool_call.id,
            "output": str(result)
        })
    
    return results

def handle_calculate(expression: str) -> str:
    """
    Safely evaluate mathematical expressions.
    """
    try:
        # Simple evaluation for basic math (in production, use a proper parser)
        # Only allow basic operators for safety
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic mathematical operations are allowed"
        
        # Evaluate the expression
        result = eval(expression)
        return f"Result: {result}"
    
    except Exception as e:
        return f"Error calculating '{expression}': {str(e)}"

def handle_get_weather(location: str, units: str = "celsius") -> str:
    """
    Mock weather function (in production, call a real weather API).
    """
    # Mock weather data
    mock_weather = {
        "seattle": {"temp": 15, "condition": "Rainy"},
        "new york": {"temp": 22, "condition": "Sunny"},
        "london": {"temp": 18, "condition": "Cloudy"},
        "tokyo": {"temp": 25, "condition": "Clear"},
        "sydney": {"temp": 20, "condition": "Partly Cloudy"}
    }
    
    location_key = location.lower()
    
    if location_key in mock_weather:
        weather = mock_weather[location_key]
        temp = weather["temp"]
        
        # Convert to Fahrenheit if requested
        if units == "fahrenheit":
            temp = (temp * 9/5) + 32
            temp_unit = "°F"
        else:
            temp_unit = "°C"
        
        return f"Weather in {location}: {weather['condition']}, {temp}{temp_unit}"
    else:
        return f"Weather data not available for {location}. Try: Seattle, New York, London, Tokyo, or Sydney"

print(" Tool handlers implemented:")
print("   - Calculator: Basic math operations")
print("   - Weather: Mock weather data for demo cities")

 Tool handlers implemented:
   - Calculator: Basic math operations
   - Weather: Mock weather data for demo cities


In [8]:
# Test tool agent with function calling
def test_tool_agent(agent, user_message: str):
    """
    Test agent with tool calling capabilities.
    """
    if not agent:
        print(" No tool agent available for testing")
        return
    
    print(f" Testing Tool Agent...")
    print(f" User: {user_message}")
    
    try:
        # Create a conversation thread
        thread = agents_client.threads.create()
        print(f" Thread created: {thread.id}")
        
        # Send message to thread
        message = agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_message
        )
        
        # Run the agent on the thread
        run = agents_client.runs.create(
            thread_id=thread.id,
            agent_id=agent.id
        )
        
        print("⏳ Agent is processing...")
        
        # Poll for completion and handle tool calls
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)
            run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
            
            # Handle tool calls if required
            if run.status == "requires_action":
                tool_calls = run.required_action.submit_tool_outputs.tool_calls
                
                # Handle the tool calls
                tool_outputs = handle_tool_calls(tool_calls)
                
                # Submit tool outputs back to the agent
                run = agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )
                
                print(" Tool outputs submitted to agent")
        
        if run.status == "completed":
            # Get the response
            messages = agents_client.messages.list(thread_id=thread.id)
            
            # Find the assistant's response (most recent message)
            message_list = list(messages)
            for msg in message_list:
                if msg.role == "assistant":
                    response_content = msg.content[0].text.value
                    print(f" Agent: {response_content}")
                    break
        else:
            print(f" Run ❌ failed with status: {run.status}")
            
        return thread.id
        
    except Exception as e:
        print(f" Tool agent test ❌ failed: {e}")
        return None

# Test the tool agent with different scenarios
if tool_agent:
    print(" Testing Tool Agent with Different Scenarios:")
    print("=" * 60)
    
    # Test 1: Calculator
    print("\n Test 1: Calculator")
    test_tool_agent(tool_agent, "Calculate the area of a circle with radius 7 (use π ≈ 3.14159)")
    
    time.sleep(2)
    
    # Test 2: Weather
    print("\n Test 2: Weather")
    test_tool_agent(tool_agent, "What's the weather like in Tokyo?")
    
    time.sleep(2)
    
    # Test 3: Complex query requiring multiple tools
    print("\n Test 3: Multi-step reasoning")
    test_tool_agent(tool_agent, "If it's 72°F in New York, what would that be in Celsius? Also calculate what 20% of 150 is.")

 Testing Tool Agent with Different Scenarios:

 Test 1: Calculator
 Testing Tool Agent...
 User: Calculate the area of a circle with radius 7 (use π ≈ 3.14159)
 Thread created: thread_lAbO1JNnkOqq8Uzzxdt9LZZh
⏳ Agent is processing...
 Agent calling tool: calculate
 Arguments: {'expression': '3.14159 * 7 * 7'}
 Tool outputs submitted to agent
 Agent: The area of a circle with radius 7, using π approximately 3.14159, is 153.93791 square units.

 Test 2: Weather
 Testing Tool Agent...
 User: What's the weather like in Tokyo?
 Thread created: thread_0BI9hKZ5MU2ZjuJDrqI53Ltj
⏳ Agent is processing...
 Agent calling tool: get_weather
 Arguments: {'location': 'Tokyo', 'units': 'celsius'}
 Tool outputs submitted to agent
 Agent: The weather in Tokyo is currently clear with a temperature of 25°C.

 Test 3: Multi-step reasoning
 Testing Tool Agent...
 User: If it's 72°F in New York, what would that be in Celsius? Also calculate what 20% of 150 is.
 Thread created: thread_OGY6roXWj3pulwWdc13gPmEZ


## 7. Advanced Agent Concepts

Now let's explore more advanced agent capabilities including specialized agents and memory management.

In [9]:
# Create specialized agents for different domains
def create_research_agent():
    """
    Create a specialized research agent.
    """
    instructions = """
    You are a research specialist AI agent. Your role is to:
    1. Analyze research topics and break them down into key components
    2. Suggest reliable sources and research methodologies
    3. Provide structured research plans
    4. Summarize findings clearly and objectively
    
    Always be thorough, cite sources when possible, and present information in a structured format.
    """
    
    try:
        agent = agents_client.create_agent(
            model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
            name="Research Specialist",
            instructions=instructions,
            description="Specialized agent for research planning and analysis"
        )
        
        print(f" Research Agent created: {agent.id}")
        return agent
    except Exception as e:
        print(f" ❌ Failed to create research agent: {e}")
        return None

def create_data_analyst_agent():
    """
    Create a specialized data analysis agent.
    """
    
    # Data analysis tools
    tools = [
        {
            "type": "function",
            "function": {
                "name": "analyze_dataset",
                "description": "Analyze a dataset and provide statistical insights",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_description": {
                            "type": "string",
                            "description": "Description of the dataset to analyze"
                        },
                        "analysis_type": {
                            "type": "string",
                            "description": "Type of analysis (descriptive, correlation, trend)",
                            "enum": ["descriptive", "correlation", "trend"]
                        }
                    },
                    "required": ["data_description", "analysis_type"]
                }
            }
        }
    ]
    
    instructions = """
    You are a data analysis specialist. Your expertise includes:
    1. Statistical analysis and interpretation
    2. Data visualization recommendations
    3. Identifying patterns and trends
    4. Providing actionable insights from data
    
    Always explain your analytical approach and provide clear, data-driven recommendations.
    """
    
    try:
        agent = agents_client.create_agent(
            model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
            name="Data Analyst",
            instructions=instructions,
            description="Specialized agent for data analysis and insights",
            tools=tools
        )
        
        print(f" Data Analyst Agent created: {agent.id}")
        return agent
    except Exception as e:
        print(f" ❌ Failed to create data analyst agent: {e}")
        return None

def create_writing_agent():
    """
    Create a specialized writing and content creation agent.
    """
    instructions = """
    You are a professional writing specialist. Your expertise includes:
    1. Creating clear, engaging content for different audiences
    2. Adapting tone and style based on context
    3. Structuring information effectively
    4. Editing and improving existing content
    
    Always consider the target audience and purpose when crafting content.
    """
    
    try:
        agent = agents_client.create_agent(
            model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
            name="Writing Specialist",
            instructions=instructions,
            description="Specialized agent for content creation and writing"
        )
        
        print(f" Writing Agent created: {agent.id}")
        return agent
    except Exception as e:
        print(f" ❌ Failed to create writing agent: {e}")
        return None

# Create specialized agents
print(" Creating Specialized Agent Team...")
print("-" * 50)

research_agent = create_research_agent()
data_analyst_agent = create_data_analyst_agent()
writing_agent = create_writing_agent()

print(f"\n Specialized Agent Team Ready!")
print(f"    Research Specialist")
print(f"    Data Analyst") 
print(f"    Writing Specialist")

 Creating Specialized Agent Team...
--------------------------------------------------
 Research Agent created: asst_3flNEYnWJUMQTBzO4nSW7xPN
 Data Analyst Agent created: asst_wcB3tyCvboqnhgFR5LJlOOIB
 Writing Agent created: asst_qQejHg1gkswvPYNwFBdlhiA6

 Specialized Agent Team Ready!
    Research Specialist
    Data Analyst
    Writing Specialist


## 8. Agent Memory and Conversation Management

Learn how to manage agent memory and maintain context across multiple interactions.

In [10]:
# Agent memory and conversation management
class AgentConversationManager:
    """
    Manage ongoing conversations with agents including memory and context.
    """
    
    def __init__(self, agents_client):
        self.agents_client = agents_client
        self.active_conversations = {}
    
    def start_conversation(self, agent, conversation_id: str = None):
        """
        Start a new conversation or resume an existing one.
        """
        if conversation_id and conversation_id in self.active_conversations:
            print(f" Resuming conversation: {conversation_id}")
            return self.active_conversations[conversation_id]
        
        # Create new conversation
        thread = self.agents_client.threads.create()
        
        conversation = {
            "agent_id": agent.id,
            "thread_id": thread.id,
            "agent_name": agent.name,
            "message_count": 0,
            "created_at": time.time()
        }
        
        conversation_id = conversation_id or f"conv_{int(time.time())}"
        self.active_conversations[conversation_id] = conversation
        
        print(f" New conversation started: {conversation_id}")
        print(f" Agent: {agent.name}")
        print(f" Thread: {thread.id}")
        
        return conversation
    
    def send_message(self, conversation_id: str, message: str, handle_tools: bool = True):
        """
        Send a message in an ongoing conversation.
        """
        if conversation_id not in self.active_conversations:
            print(f" Conversation {conversation_id} not found")
            return None
        
        conversation = self.active_conversations[conversation_id]
        
        print(f"\n Conversation: {conversation_id}")
        print(f" User: {message}")
        
        try:
            # Add message to thread
            self.agents_client.messages.create(
                thread_id=conversation["thread_id"],
                role="user",
                content=message
            )
            
            # Run agent
            run = self.agents_client.runs.create(
                thread_id=conversation["thread_id"],
                agent_id=conversation["agent_id"]
            )
            
            print("⏳ Agent is processing...")
            
            # Handle the run and any tool calls
            while run.status in ["queued", "in_progress", "requires_action"]:
                time.sleep(1)
                run = self.agents_client.runs.get(
                    thread_id=conversation["thread_id"], 
                    run_id=run.id
                )
                
                if run.status == "requires_action" and handle_tools:
                    tool_calls = run.required_action.submit_tool_outputs.tool_calls
                    tool_outputs = handle_tool_calls(tool_calls)
                    
                    run = self.agents_client.runs.submit_tool_outputs(
                        thread_id=conversation["thread_id"],
                        run_id=run.id,
                        tool_outputs=tool_outputs
                    )
            
            if run.status == "completed":
                # Get latest response
                messages = self.agents_client.messages.list(thread_id=conversation["thread_id"])
                
                message_list = list(messages)
                for msg in message_list:
                    if msg.role == "assistant":
                        response = msg.content[0].text.value
                        print(f" {conversation['agent_name']}: {response}")
                        
                        # Update conversation stats
                        conversation["message_count"] += 1
                        conversation["last_activity"] = time.time()
                        
                        return response
            else:
                print(f" Run ❌ failed with status: {run.status}")
                return None
                
        except Exception as e:
            print(f" ❌ Failed to send message: {e}")
            return None
    
    def get_conversation_history(self, conversation_id: str):
        """
        Get the full conversation history.
        """
        if conversation_id not in self.active_conversations:
            print(f" Conversation {conversation_id} not found")
            return []
        
        conversation = self.active_conversations[conversation_id]
        
        try:
            messages = self.agents_client.messages.list(thread_id=conversation["thread_id"])
            
            history = []
            message_list = list(messages)
            for msg in reversed(message_list):  # Reverse to get chronological order
                history.append({
                    "role": msg.role,
                    "content": msg.content[0].text.value,
                    "timestamp": msg.created_at
                })
            
            return history
            
        except Exception as e:
            print(f" ❌ Failed to get conversation history: {e}")
            return []
    
    def list_conversations(self):
        """
        List all active conversations.
        """
        print(" Active Conversations:")
        print("-" * 50)
        
        for conv_id, conv in self.active_conversations.items():
            duration = time.time() - conv["created_at"]
            print(f" {conv_id}")
            print(f"    Agent: {conv['agent_name']}")
            print(f"    Messages: {conv['message_count']}")
            print(f"   ⏱ Duration: {duration:.0f} seconds")
            print("")

# Initialize conversation manager
if 'agents_client' in globals():
    conversation_manager = AgentConversationManager(agents_client)
    print(" Conversation Manager initialized")
    print("    Start conversations with start_conversation()")
    print("    Send messages with send_message()")
    print("    View history with get_conversation_history()")
else:
    print(" Agents client not available")

 Conversation Manager initialized
    Start conversations with start_conversation()
    Send messages with send_message()
    View history with get_conversation_history()


In [11]:
# Test conversation memory and continuity
def demo_conversation_memory():
    """
    Demonstrate agent memory and conversation continuity.
    """
    if not tool_agent:
        print(" Tool agent not available for memory demo")
        return
    
    print(" Testing Agent Memory and Conversation Continuity")
    print("=" * 60)
    
    # Start a conversation
    conversation = conversation_manager.start_conversation(tool_agent, "memory_demo")
    
    # First interaction - establish context
    print("\n Step 1: Establishing context")
    conversation_manager.send_message(
        "memory_demo", 
        "Hi! I'm planning a trip to Seattle. Can you help me with some calculations and information?"
    )
    
    time.sleep(2)
    
    # Second interaction - use established context
    print("\n Step 2: Building on context")
    conversation_manager.send_message(
        "memory_demo",
        "What's the weather like there? And if my budget is $1200, what's 15% of that for tips and extras?"
    )
    
    time.sleep(2)
    
    # Third interaction - reference previous conversation
    print("\n Step 3: Referencing previous discussion")
    conversation_manager.send_message(
        "memory_demo",
        "Based on our conversation about my Seattle trip, can you calculate how much I'd have left after the 15% we just calculated?"
    )
    
    # Show conversation history
    print("\n Full Conversation History:")
    print("-" * 40)
    history = conversation_manager.get_conversation_history("memory_demo")
    
    for i, msg in enumerate(history, 1):
        role_emoji = "" if msg["role"] == "user" else ""
        print(f"{i}. {role_emoji} {msg['role'].title()}: {msg['content'][:100]}...")
        print("")

# Run the memory demonstration
if 'conversation_manager' in globals() and tool_agent:
    demo_conversation_memory()
else:
    print(" Create agents and conversation manager first to test memory")

 Testing Agent Memory and Conversation Continuity
 New conversation started: memory_demo
 Agent: Tool Agent
 Thread: thread_vg1Ro6Cq7SuYTBWpo2iWAQVp

 Step 1: Establishing context

 Conversation: memory_demo
 User: Hi! I'm planning a trip to Seattle. Can you help me with some calculations and information?
⏳ Agent is processing...
 Tool Agent: Hello! I'd be happy to help you with your trip to Seattle. What kind of calculations or information do you need? Weather, distances, event timings, or something else? Let me know!

 Step 2: Building on context

 Conversation: memory_demo
 User: What's the weather like there? And if my budget is $1200, what's 15% of that for tips and extras?
⏳ Agent is processing...
 Agent calling tool: get_weather
 Arguments: {'location': 'Seattle', 'units': 'celsius'}
 Agent calling tool: calculate
 Arguments: {'expression': '1200 * 0.15'}
 Tool Agent: The weather in Seattle is currently rainy with a temperature of 15°C. 

For your budget of $1200, 15% allocated 

## 9. Monitoring Agent Interactions

Learn how to monitor, and debug agent interactions for production use.

In [12]:
# Agent monitoring and analytics
class AgentMonitor:
    """
    Monitor and analyze agent interactions for insights and debugging.
    """
    
    def __init__(self):
        self.interactions = []
        self.performance_metrics = {}
    
    def log_interaction(self, agent_name: str, user_message: str, 
                       agent_response: str, tools_used: list = None, 
                       response_time: float = 0, success: bool = True):
        """
        Log an agent interaction for monitoring.
        """
        interaction = {
            "timestamp": time.time(),
            "agent_name": agent_name,
            "user_message": user_message,
            "agent_response": agent_response,
            "tools_used": tools_used or [],
            "response_time": response_time,
            "success": success,
            "message_length": len(user_message),
            "response_length": len(agent_response)
        }
        
        self.interactions.append(interaction)
    
    def get_agent_statistics(self, agent_name: str = None):
        """
        Get performance statistics for agents.
        """
        filtered_interactions = self.interactions
        if agent_name:
            filtered_interactions = [i for i in self.interactions if i["agent_name"] == agent_name]
        
        if not filtered_interactions:
            print(f" No interactions found{' for ' + agent_name if agent_name else ''}")
            return
        
        total_interactions = len(filtered_interactions)
        successful_interactions = len([i for i in filtered_interactions if i["success"]])
        success_rate = (successful_interactions / total_interactions) * 100
        
        avg_response_time = sum(i["response_time"] for i in filtered_interactions) / total_interactions
        
        tools_usage = {}
        for interaction in filtered_interactions:
            for tool in interaction["tools_used"]:
                tools_usage[tool] = tools_usage.get(tool, 0) + 1
        
        print(f" Agent Statistics{' for ' + agent_name if agent_name else ' (All Agents)'}")
        print("-" * 50)
        print(f" Total Interactions: {total_interactions}")
        print(f" Success Rate: {success_rate:.1f}%")
        print(f"⏱ Average Response Time: {avg_response_time:.2f}s")
        
        if tools_usage:
            print(f"\n Tools Usage:")
            for tool, count in sorted(tools_usage.items(), key=lambda x: x[1], reverse=True):
                print(f"   {tool}: {count} times")
        
        return {
            "total_interactions": total_interactions,
            "success_rate": success_rate,
            "avg_response_time": avg_response_time,
            "tools_usage": tools_usage
        }
    
    def get_recent_interactions(self, limit: int = 5):
        """
        Get recent agent interactions.
        """
        recent = sorted(self.interactions, key=lambda x: x["timestamp"], reverse=True)[:limit]
        
        print(f" Recent {min(limit, len(recent))} Interactions:")
        print("-" * 50)
        
        for i, interaction in enumerate(recent, 1):
            timestamp = time.strftime("%H:%M:%S", time.localtime(interaction["timestamp"]))
            status = "" if interaction["success"] else ""
            tools = f" [Tools: {', '.join(interaction['tools_used'])}]" if interaction["tools_used"] else ""
            
            print(f"{i}. {status} {timestamp} - {interaction['agent_name']}")
            print(f"    User: {interaction['user_message'][:60]}...")
            print(f"    Agent: {interaction['agent_response'][:60]}...{tools}")
            print(f"   ⏱ {interaction['response_time']:.2f}s")
            print("")
    
    def analyze_patterns(self):
        """
        Analyze patterns in agent interactions.
        """
        if not self.interactions:
            print(" No interactions to analyze")
            return
        
        print(" Agent Interaction Pattern Analysis")
        print("=" * 50)
        
        # Agent usage distribution
        agent_usage = {}
        for interaction in self.interactions:
            agent_name = interaction["agent_name"]
            agent_usage[agent_name] = agent_usage.get(agent_name, 0) + 1
        
        print("\n Agent Usage Distribution:")
        for agent, count in sorted(agent_usage.items(), key=lambda x: x[1], reverse=True):
            percentage = (count / len(self.interactions)) * 100
            print(f"   {agent}: {count} interactions ({percentage:.1f}%)")
        
        # Tool usage patterns
        all_tools = []
        for interaction in self.interactions:
            all_tools.extend(interaction["tools_used"])
        
        if all_tools:
            from collections import Counter
            tool_counts = Counter(all_tools)
            
            print(f"\n Most Used Tools:")
            for tool, count in tool_counts.most_common(5):
                print(f"   {tool}: {count} times")
        
        # Response time analysis
        response_times = [i["response_time"] for i in self.interactions if i["response_time"] > 0]
        if response_times:
            avg_time = sum(response_times) / len(response_times)
            max_time = max(response_times)
            min_time = min(response_times)
            
            print(f"\n⏱ Response Time Analysis:")
            print(f"   Average: {avg_time:.2f}s")
            print(f"   Fastest: {min_time:.2f}s")
            print(f"   Slowest: {max_time:.2f}s")

# Initialize agent monitor
agent_monitor = AgentMonitor()

print(" Agent Monitor initialized")
print("    Log interactions with log_interaction()")
print("    View statistics with get_agent_statistics()")
print("    Analyze patterns with analyze_patterns()")

 Agent Monitor initialized
    Log interactions with log_interaction()
    View statistics with get_agent_statistics()
    Analyze patterns with analyze_patterns()


## Workshop Summary

Congratulations! You've completed a comprehensive workshop on AI Agents with Azure AI Foundry.

### What You've Accomplished

1.  **Agent Fundamentals** - Understood AI agent capabilities and architecture
2.  **Azure AI Agent Service** - Connected to and used the cloud-native agent platform
3.  **Basic Agent Creation** - Built your first AI agent with simple conversations
4.  **Function Calling** - Created agents with custom tools (calculator, weather)
5.  **Tool Implementation** - Built tool handlers for agent function calls
6.  **Specialized Agents** - Created domain-specific agents (Research, Data Analysis, Writing)
7.  **Memory Management** - Implemented conversation continuity and context retention
8.  **Monitoring & Analytics** - Added comprehensive agent interaction tracking

### Key Concepts Mastered

- **Agent Architecture**: Understanding autonomy, planning, and tool use
- **Tool Integration**: Creating and managing custom agent capabilities
- **Conversation Management**: Handling stateful, multi-turn interactions
- **Multi-Agent Systems**: Coordinating specialized agents for complex tasks
- **Performance Monitoring**: Tracking and optimizing agent interactions
- **Production Considerations**: Security, error handling, and scalability

### Production Readiness Checklist

Before deploying agents to production, consider:

- [ ] **Security**: Validate all tool inputs and sanitize outputs
- [ ] **Error Handling**: Implement robust error recovery and fallbacks
- [ ] **Rate Limiting**: Control agent usage and API call frequency
- [ ] **Monitoring**: Set up comprehensive logging and alerting
- [ ] **Testing**: Create thorough test suites for agent behavior
- [ ] **Documentation**: Document agent capabilities and limitations
- [ ] **Compliance**: Ensure data handling meets regulatory requirements

### Advanced Topics to Explore Next

1. **Agent Orchestration**: Building complex multi-agent workflows
2. **Custom Models**: Fine-tuning models for specific agent tasks
3. **Integration Patterns**: Connecting agents to enterprise systems
4. **Scaling Strategies**: Handling high-volume agent interactions
5. **Security Frameworks**: Implementing enterprise-grade agent security
6. **Cost Optimization**: Efficient resource usage and model selection

### Real-World Applications

Consider these practical use cases for your AI agents:

- **Customer Support**: Automated ticket handling and resolution
- **Content Creation**: Research, writing, and editing assistance
- **Data Analysis**: Automated insights and reporting
- **Process Automation**: Workflow orchestration and task management
- **Decision Support**: Analytical reasoning and recommendation systems

### Continue Learning

- **Azure AI Foundry Documentation**: Explore advanced agent features
- **OpenAI Function Calling**: Deepen your tool integration knowledge
- **Multi-Agent Frameworks**: Study collaboration patterns
- **AI Safety**: Learn responsible AI agent development practices
